In [4]:
# 環境初始化
import json
import pickle

import networkx as nx
from langchain_openai import ChatOpenAI
from graph_pipeline.embeding_entity_relation_to_pincone import jina_embedding
from pinecone import Pinecone

# 載入知識圖譜
with open('data/knowledge_graph.pkl', 'rb') as f:
    G = pickle.load(f)

# 載入 env 設定
with open('env.json', 'r', encoding='utf-8') as file:
    config = json.load(file)


# 初始化 OpenAI 模型
llm = ChatOpenAI(
    model="gpt-3.5-turbo", 
    openai_api_key=config["chat_gpt_key"],
    temperature=0.6
)

c:\Users\TMP214\Desktop\GitHub\project_RAG_and_Graph\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
# 對 llm 提問
def query_llm(prompt):
    try:
        response = llm.invoke(prompt)
        return response.content
    except:
        return f"LLM 錯誤"


# LLM 問題實體提取
def query_entity_extract(prompt):
    prompt_template = f'''
    # 實體與關係知識圖譜識別任務

    ## 輸入文本
    ```
    {prompt}
    ```

    ## 任務說明
    1. 從上述文本中識別所有概念
    2. 將識別的概念映射到實體限制列表中的具體實體（而非分類標題）
    3. 識別文本中提及的關係，只使用關係限制列表中的關係，且至少一個關係

    ## 實體限制
    - **霉菌類型**：青霉菌、黑曲霉、毛霉菌、木霉菌、根霉菌、白色霉菌、粉紅霉菌、灰綠霉菌、交鏈孢菌、褐腐菌
    - **鞋類材質**：天然皮革、合成皮革、帆布、麂皮、絨面革、尼龍網面、PVC材質、橡膠、EVA泡棉、彈性紡織物、專利皮革、亞麻材質、牛津布、棉質材料、羊毛毛氈、人造絨面、尼龍與聚酯混紡、防水透氣膜、真絲材質、特氟龍塗層材質
    - **鞋類種類**：皮鞋、運動鞋、帆布鞋、涼鞋、拖鞋、靴子、高跟鞋、休閒鞋、兒童鞋、跑步鞋、籃球鞋、足球鞋、登山鞋、室內鞋、船鞋、工作鞋、芭蕾舞鞋、雨靴、收藏級限量版鞋、定制鞋
    - **環境條件**：高濕度、中等濕度、低濕度、熱帶氣候、亞熱帶氣候、溫帶氣候、海洋性氣候、大陸性氣候、雨季環境、季風季節、室內空調環境、浴室環境、地下室環境、密閉鞋櫃、開放式鞋架、陰暗儲藏室、行李箱內環境、塑料袋封存環境、專業恆溫恆濕倉庫、紙盒包裝環境
    - **霉變症狀**：白色粉末狀斑點、綠色絨毛狀附著物、黑色霉斑、灰色斑點、黃褐色變色、霉味、表面粗糙、材質軟化、纖維分解、色彩褪變、斑駁痕跡、邊緣霉變、內襯發霉、鞋墊霉變、縫線處霉變、全面性霉變、點狀霉斑、絲狀霉菌蔓延、顏色暗沉、結構性損壞
    - **防霉產品**：70% 酒精溶液、白醋稀釋液、小蘇打溶液、硼酸鹽防霉劑、四級銨鹽防霉劑、茶樹油噴霧、含銀離子防霉劑、矽膠乾燥劑、活性炭吸濕包、鈣氯乾燥劑、商業防霉噴霧、皮革專用防霉油、防霉膏、除濕盒、防霉貼片、電子除濕器、UV殺菌燈、臭氧發生器、納米防霉塗層、防霉抗菌鞋墊
    - **清潔工具**：軟毛刷、麂皮專用刷、奶油刷、舊牙刷、微纖維布、棉布、海綿、清潔棒、噴霧瓶、紙巾、電動清潔刷、蒸汽清潔機、冷風吹風機、吸塵器附件、手套、口罩、防護眼鏡、橡膠刮刀、棉簽、擦鞋巾
    - **儲存設備**：透氣鞋盒、防霉鞋袋、雪松木鞋楦、木製鞋架、金屬鞋架、懸掛式鞋袋、除濕鞋櫃、智能鞋櫃、真空壓縮袋、塑料儲存箱、無酸紙、防潮箱、恆溫恆濕櫃、電子防霉鞋架、通風鞋架、抽屜式鞋箱、掛壁式鞋架、旋轉鞋架、布質鞋套、專業展示架
    - **霉變程度分類**：輕度霉變、中度霉變、重度霉變、極重度霉變、初期霉變、表面霉變、深層霉變、全面霉變、局部霉變、反復霉變
    - **健康風險**：過敏反應、呼吸道刺激、皮膚接觸性過敏、氣喘惡化、霉菌孢子吸入、霉菌毒素接觸、免疫功能低下者風險、兒童暴露風險、眼部刺激、接觸性皮膚炎、慢性暴露引起的健康問題、職業暴露風險、交叉污染風險、敏感人群的特殊風險、寵物健康風險
    - **專業服務**：專業鞋類清潔服務、皮革修復專家、收藏鞋防霉處理、批量商業防霉處理、霉變評估服務、鞋類修復工作室、博物館級文物保護、零售商庫存防霉服務、霉菌檢測實驗室、環境控制顧問、專業除濕服務、防霉系統安裝、皮鞋翻新服務、定制鞋保養服務、奢侈品保養專家、霉變保險評估、商業空間霉菌控制、環保型霉菌處理、防霉技術培訓、鞋類壽命評估

    ## 關係限制
    生長於、需要條件、活躍於、優勢於、感染、偏好、常見於、導致、引起、表現為、受影響、易感、抵抗、傳播通過、預防使用、清除使用、殺滅、適用於、存放於、損害、加速、延緩、檢測通過、發生於、維護使用、清潔使用、影響、評估使用、保養使用、控制使用、監測使用、應對、處理於、費用為、持續時間、適合、不適合、識別通過、保存於、按標準、提供服務、使用工具、發展為、降低風險、增加風險、分類為、證明通過、相關知識、協同作用、對抗、替代、補充、運輸通過、存在於、由引起、源自、保護、修復、回復、加工使用、管理通過、投資回報、發明於、研發由、結合、分解、更新頻率、適用氣候、安全級別、效果持續、成本比、風險等級、處理難度、應用於、轉化為、受規範、影響程度、特性為、分佈於、生命週期、屬於、包含、作用於、反應於、承受、使用場景、由組成、專門針對、必要條件、充分條件、可替代、不可替代、產生於、消耗


    ## 輸出格式
    {{
        "實體": [列出所有從文本中識別出的具體實體名稱],
        "關係": [列出所有從文本中識別出的關係，只使用關係限制列表中的關係]
    }}
    '''
    return query_llm(prompt_template)


# 將實體與關係用合理的方式組合起來
def entity_relation_combine(prompt):
    prompt_template = f'''
    將實體與關係用合理的方式組合成知識圖譜

    ## 輸入資料 : 
    {prompt}

    ## 指導原則
    1. source 與 target 只能用 '實體' 內的元素
    2. relation 只能用 '關係' 內的元素

    ## 輸出資料 : 
    請以以下JSON格式返回知識圖譜：
    [
        {{"source": "<實體>", "target": "<實體>", "relation": "<關係>"}},...
    ]
    '''
    return query_llm(prompt_template)


# 將知識圖譜出來的數據與使用者的問題給出說明
def query_llm_graph_RAG(query, graph_prompt):
    prompt = f'''
        請按照知識圖譜出來的數據與使用者的問題給出說明

        1. 使用者問題:
        {query}

        2. 知識圖譜數據
        {graph_prompt}
    '''

    return query_llm(prompt)


In [6]:
# 由向量化搜尋實體限制列表前三個適合的
def serch_embedding(index_name, text, config):
    '''
    放入 text 回傳 3 個在 pinecone index 內最接近的 限制列表
    '''
    pc = Pinecone(api_key=config['pinecone_api_key'])
    query_embedding = jina_embedding(text, config)
    index = pc.Index(index_name)
    results = index.query(
        vector=query_embedding,
        top_k=3,
        include_metadata=True
    )

    matches = results['matches']
    matches_list = []
    for match in matches:
        matches_list.append(match['metadata']['text'])
    return matches_list

In [ ]:
# 由 使用者問題通過 llm 提取關係實體過程
def extract_entity_and_relation(prompt):
    # 1. llm 提取關係實體
    data = query_entity_extract(prompt)
    data = json.loads(data)
    print(data)


    # 2. 將關係與實體 list 每個都各搜尋向量資料庫 top3 相近的圖譜內實體與關係
    print('=========================')
    entity_list = []
    for text in data["實體"]:
        entity_list += serch_embedding('entity', text, config)
    entity_list = list(set(entity_list))  # 去重複
    print(entity_list)

    relation_list = []
    for text in data["關係"]:
        relation_list += serch_embedding('relation', text, config)
    relation_list = list(set(relation_list))
    print(relation_list)

    graph_dict = {
        "實體" : entity_list,
        "關係" : relation_list
    }
    print(graph_dict)
    print('===========================')

    # 3. 將分散的關係與實體重新通過 llm 組成 (source, target, relation) 組合
    graph_dict = json.loads(entity_relation_combine(graph_dict))
    print(graph_dict)

    return graph_dict

In [8]:
# 建立查詢圖譜功能
def graph_query(G, source, target, relation=None):
    """
    查詢從source到target的路徑，優先考慮包含指定關係的路徑
    
    Args:
        G (nx.DiGraph): 知識圖譜
        source (str): 起始節點
        target (str): 目標節點
        relation (str, optional): 優先選擇的關係類型
    
    Returns:
        list: 包含路徑的列表，每個元素為(source, target, relation, description)的元組
    """
    # 檢查節點是否存在
    if source not in G.nodes():
        print(f"錯誤: 來源節點 '{source}' 不存在於圖譜中")
        return []
    
    if target not in G.nodes():
        print(f"錯誤: 目標節點 '{target}' 不存在於圖譜中")
        return []
    
    # 初始化結果列表
    path_info = []
    
    # 嘗試尋找所有路徑
    try:
        # 如果指定了關係，優先尋找包含該關係的路徑
        if relation:
            # 建立一個權重圖，使得含有指定關係的邊權重較小
            weighted_G = G.copy()
            for u, v, data in G.edges(data=True):
                if data.get('relation') == relation:
                    weighted_G[u][v]['weight'] = 0.1  # 優先選擇的關係權重小
                else:
                    weighted_G[u][v]['weight'] = 1.0
                    
            # 使用Dijkstra算法查找最短路徑
            path = nx.shortest_path(weighted_G, source=source, target=target, weight='weight')
            
            # 提取路徑信息
            for i in range(len(path) - 1):
                s, t = path[i], path[i + 1]
                rel = G[s][t].get('relation', '')
                desc = G[s][t].get('description', '')
                path_info.append((s, t, rel, desc))
                
        else:
            # 如果未指定關係，直接尋找最短路徑
            path = nx.shortest_path(G, source=source, target=target)
            
            # 提取路徑信息
            for i in range(len(path) - 1):
                s, t = path[i], path[i + 1]
                rel = G[s][t].get('relation', '')
                desc = G[s][t].get('description', '')
                path_info.append((s, t, rel, desc))
                
    except nx.NetworkXNoPath:
        print(f"無法從 '{source}' 到 '{target}' 找到路徑")
    except nx.NetworkXError as e:
        print(f"發生錯誤: {e}")
    
    # 印出結果
    if path_info:
        print(f"從 '{source}' 到 '{target}' 的路徑:")
        for s, t, rel, desc in path_info:
            print(f"  {s} --[{rel}]--> {t}  (描述: {desc})")
    else:
        print(f"未找到從 '{source}' 到 '{target}' 的路徑")
    
    return path_info

In [9]:
def query_graph_RAG(query):
    # 1. 從 query 內提出實體與關係
    graph_dict = extract_entity_and_relation(query)

    # 2. 找出知識圖譜中所有可能的實體關係路徑與描述
    path_info_list = []
    for dict in graph_dict:
        path_info = graph_query(G, dict["source"], dict['target'], dict['relation'])
        path_info_list.append(path_info)

    print('===========================')
    print(path_info_list)

    # 3. 根據 使用者問題 與 知識圖譜 描述回答問題
    print('=======回答=====================')
    answer = query_llm_graph_RAG(query, path_info_list)
    print(answer)

In [ ]:
# 1. 基礎事實查詢 - 最簡單
query_graph_RAG('鞋子發霉怎麼辦？')

{'實體': ['鞋子', '發霉', '霉菌類型', '環境條件', '霉變症狀', '防霉產品', '清潔工具'], '關係': ['存放於', '影響', '清潔使用']}
['初期霉變', '表面霉變', '批量商業防霉處理', '霉味', '防霉抗菌鞋墊', '電動清潔刷', '中度霉變', '輕度霉變', '青霉菌', '毛霉菌', '中等濕度', '內襯發霉', '環境控制顧問', '防霉貼片', '軟毛刷', '休閒鞋', '運動鞋', '根霉菌', '高濕度', '舊牙刷', '靴子']
['保護', '維護使用', '存放於', '保養使用', '影響', '清潔使用', '導致', '保存於', '受影響']
{'實體': ['初期霉變', '表面霉變', '批量商業防霉處理', '霉味', '防霉抗菌鞋墊', '電動清潔刷', '中度霉變', '輕度霉變', '青霉菌', '毛霉菌', '中等濕度', '內襯發霉', '環境控制顧問', '防霉貼片', '軟毛刷', '休閒鞋', '運動鞋', '根霉菌', '高濕度', '舊牙刷', '靴子'], '關係': ['保護', '維護使用', '存放於', '保養使用', '影響', '清潔使用', '導致', '保存於', '受影響']}
[{'source': '初期霉變', 'target': '中度霉變', 'relation': '影響'}, {'source': '表面霉變', 'target': '批量商業防霉處理', 'relation': '保護'}, {'source': '批量商業防霉處理', 'target': '防霉抗菌鞋墊', 'relation': '保護'}, {'source': '霉味', 'target': '防霉抗菌鞋墊', 'relation': '導致'}, {'source': '電動清潔刷', 'target': '清潔使用', 'relation': '保養使用'}, {'source': '中度霉變', 'target': '內襯發霉', 'relation': '受影響'}, {'source': '輕度霉變', 'target': '中度霉變', 'relation': '影響'}, {'source': '青霉菌', 'target': 

In [ ]:
# 2. 比較查詢 - 中等複雜度
query_graph_RAG('白色和黑色黴菌對鞋子的危害有什麼不同？')

{'實體': ['白色霉菌', '黑色霉菌', '鞋子'], '關係': ['損害']}
['休閒鞋', '黑色霉斑', '白色霉菌', '運動鞋', '灰綠霉菌', '褐腐菌', '粉紅霉菌', '青霉菌', '靴子']
['損害', '受影響', '導致']
{'實體': ['休閒鞋', '黑色霉斑', '白色霉菌', '運動鞋', '灰綠霉菌', '褐腐菌', '粉紅霉菌', '青霉菌', '靴子'], '關係': ['損害', '受影響', '導致']}
[{'source': '休閒鞋', 'target': '黑色霉斑', 'relation': '受影響'}, {'source': '休閒鞋', 'target': '白色霉菌', 'relation': '受影響'}, {'source': '運動鞋', 'target': '灰綠霉菌', 'relation': '受影響'}, {'source': '運動鞋', 'target': '褐腐菌', 'relation': '受影響'}, {'source': '運動鞋', 'target': '粉紅霉菌', 'relation': '受影響'}, {'source': '靴子', 'target': '青霉菌', 'relation': '受影響'}]
從 '休閒鞋' 到 '黑色霉斑' 的路徑:
  休閒鞋 --[維護使用]--> 開放式鞋架  (描述: 每週定期將鞋子取出通風幾小時)
  開放式鞋架 --[對抗]--> 高濕度  (描述: 通風改善：確保儲存區域有良好的空氣流通)
  高濕度 --[協同作用]--> 青霉菌  (描述: 溫度與濕度協同作用)
  青霉菌 --[導致]--> 黑色霉斑  (描述: 美觀問題：霉斑和變色)
從 '休閒鞋' 到 '白色霉菌' 的路徑:
  休閒鞋 --[維護使用]--> 開放式鞋架  (描述: 每週定期將鞋子取出通風幾小時)
  開放式鞋架 --[降低風險]--> 輕度霉變  (描述: 開放式鞋架：促進通風，降低霉變風險)
  輕度霉變 --[表現為]--> 白色粉末狀斑點  (描述: 輕度：表面有少量霉點，無明顯氣味)
  白色粉末狀斑點 --[特性為]--> 白色霉菌  (描述: 外觀：絨毛/粉狀)
從 '運動鞋' 到 '灰綠霉菌' 的路徑:
  運動鞋

In [ ]:
# 3. 多步驟處理查詢 - 中高複雜度
query_graph_RAG('我的運動鞋裡外都有黴菌，怎麼徹底清洗並防止再次發霉？')

{'實體': ['霉菌類型', '鞋類材質', '清潔工具', '防霉產品'], '關係': ['清除使用', '預防使用']}
['毛霉菌', '室內鞋', '批量商業防霉處理', '運動鞋', '防霉抗菌鞋墊', '電動清潔刷', '根霉菌', '青霉菌', '軟毛刷', '防霉貼片', '舊牙刷', '靴子']
['降低風險', '清除使用', '處理於', '保養使用', '清潔使用', '預防使用']
{'實體': ['毛霉菌', '室內鞋', '批量商業防霉處理', '運動鞋', '防霉抗菌鞋墊', '電動清潔刷', '根霉菌', '青霉菌', '軟毛刷', '防霉貼片', '舊牙刷', '靴子'], '關係': ['降低風險', '清除使用', '處理於', '保養使用', '清潔使用', '預防使用']}
[{'source': '毛霉菌', 'target': '室內鞋', 'relation': '降低風險'}, {'source': '毛霉菌', 'target': '批量商業防霉處理', 'relation': '處理於'}, {'source': '室內鞋', 'target': '運動鞋', 'relation': '保養使用'}, {'source': '室內鞋', 'target': '防霉抗菌鞋墊', 'relation': '保養使用'}, {'source': '室內鞋', 'target': '電動清潔刷', 'relation': '清潔使用'}, {'source': '批量商業防霉處理', 'target': '根霉菌', 'relation': '預防使用'}, {'source': '批量商業防霉處理', 'target': '青霉菌', 'relation': '預防使用'}, {'source': '電動清潔刷', 'target': '軟毛刷', 'relation': '清潔使用'}, {'source': '防霉抗菌鞋墊', 'target': '防霉貼片', 'relation': '保養使用'}, {'source': '軟毛刷', 'target': '舊牙刷', 'relation': '清潔使用'}, {'source': '靴子', 'target': '根霉菌', 'relation': '降

In [18]:
# 4. 專業知識與健康關聯查詢 - 高複雜度
query_graph_RAG('長期穿著發霉的鞋子對足部健康有哪些特定風險？特別是對糖尿病患者來說。')

{'實體': ['鞋子', '足部健康', '糖尿病患者', '發霉的鞋子', '健康風險'], '關係': ['影響', '對抗']}
['白色粉末狀斑點', '運動鞋', '鞋墊霉變', '靴子', '極重度霉變', '內襯發霉', '絲狀霉菌蔓延', '敏感人群的特殊風險', '中度霉變', '休閒鞋']
['對抗', '影響', '抵抗', '導致', '反應於', '受影響']
{'實體': ['白色粉末狀斑點', '運動鞋', '鞋墊霉變', '靴子', '極重度霉變', '內襯發霉', '絲狀霉菌蔓延', '敏感人群的特殊風險', '中度霉變', '休閒鞋'], '關係': ['對抗', '影響', '抵抗', '導致', '反應於', '受影響']}
[{'source': '白色粉末狀斑點', 'target': '運動鞋', 'relation': '對抗'}, {'source': '鞋墊霉變', 'target': '運動鞋', 'relation': '影響'}, {'source': '靴子', 'target': '極重度霉變', 'relation': '抵抗'}, {'source': '內襯發霉', 'target': '靴子', 'relation': '導致'}, {'source': '絲狀霉菌蔓延', 'target': '內襯發霉', 'relation': '反應於'}, {'source': '敏感人群的特殊風險', 'target': '絲狀霉菌蔓延', 'relation': '受影響'}, {'source': '中度霉變', 'target': '休閒鞋', 'relation': '對抗'}]
從 '白色粉末狀斑點' 到 '運動鞋' 的路徑:
  白色粉末狀斑點 --[特性為]--> 白色霉菌  (描述: 外觀：絨毛/粉狀)
  白色霉菌 --[需要條件]--> 高濕度  (描述: 在適宜的環境條件下，這些孢子會迅速生長並形成可見的霉菌菌落)
  高濕度 --[損害]--> 運動鞋  (描述: 運輸和儲存中的霉變損失)
無法從 '鞋墊霉變' 到 '運動鞋' 找到路徑
未找到從 '鞋墊霉變' 到 '運動鞋' 的路徑
無法從 '靴子' 到 '極重度霉變' 找到路徑
未找到從 '靴子' 到 '極重度霉變' 的路徑

In [ ]:
# 5. 情境分析與多方案比較查詢 - 最高複雜度
query_graph_RAG('我在熱帶高濕地區工作，需要長期在戶外穿著皮靴，已經出現反覆發霉問題。請比較不同防黴方案的效果、成本和實用性，並給出最適合我情況的解決方案。')

{'實體': ['熱帶高濕地區', '皮靴', '反覆發霉問題', '防黴方案', '效果', '成本', '實用性'], '關係': ['適合', '持續時間', '成本比']}
['表面霉變', '批量商業防霉處理', '皮鞋', '溫帶氣候', '表面粗糙', '亞熱帶氣候', '防霉技術培訓', '反復霉變', '白色粉末狀斑點', '防霉系統安裝', '雨靴', '極重度霉變', '內襯發霉', '材質軟化', '紙巾', '軟毛刷', '霉變保險評估', '熱帶氣候', '霉變評估服務', '靴子']
['常見於', '評估使用', '費用為', '消耗', '效果持續', '適合', '延緩', '適用於', '更新頻率']
{'實體': ['表面霉變', '批量商業防霉處理', '皮鞋', '溫帶氣候', '表面粗糙', '亞熱帶氣候', '防霉技術培訓', '反復霉變', '白色粉末狀斑點', '防霉系統安裝', '雨靴', '極重度霉變', '內襯發霉', '材質軟化', '紙巾', '軟毛刷', '霉變保險評估', '熱帶氣候', '霉變評估服務', '靴子'], '關係': ['常見於', '評估使用', '費用為', '消耗', '效果持續', '適合', '延緩', '適用於', '更新頻率']}
[{'source': '表面霉變', 'target': '批量商業防霉處理', 'relation': '常見於'}, {'source': '皮鞋', 'target': '溫帶氣候', 'relation': '適合'}, {'source': '皮鞋', 'target': '表面粗糙', 'relation': '適合'}, {'source': '防霉技術培訓', 'target': '反復霉變', 'relation': '評估使用'}, {'source': '白色粉末狀斑點', 'target': '防霉系統安裝', 'relation': '消耗'}, {'source': '雨靴', 'target': '極重度霉變', 'relation': '常見於'}, {'source': '雨靴', 'target': '內襯發霉', 'relation': '常見於'}, {'source': '雨靴', 'target':